ORGANIZACIJA:
evo linka od prezentacije:
https://fb.watch/4y-LKdA7iT/

UPUTE:
pokrenuti zakomentiranu skriptu ispod i uploadat si pkl file na google disk.

In [ ]:
# import pickle

# start_time = time.time()
# tcurrent = start_time
# np.random.seed(33)

# filename = 'LUMEN_DS.csv'
# df = pd.read_csv(filename, encoding="UTF-16LE", sep='|', quotechar='"', low_memory=False)

# to_drop_too_much_missing = ['Price last modified date in the ERP',
#                             'Sales Channel - Grouping',
#                             'Material cost of part',
#                             'Labor cost of part',
#                             'Overhead cost of part']

# df.drop(to_drop_too_much_missing, inplace=True, axis=1)

# to_drop_unimportant = [
#     'Sales Channel - Internal',
#     'Sales Channel - External']

# df.drop(to_drop_unimportant, inplace=True, axis=1)

# with open('LUMEN2.pkl', 'wb') as f:
#     pickle.dump(df, f)

In [ ]:
!pip install sweetviz

In [ ]:
import pickle
import matplotlib.pyplot as plt
from google.colab import drive
import os
import time
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
import sweetviz as sv
import xgboost as xgb
import pickle
from sklearn import metrics
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = "/content/drive/MyDrive/LUMEN0.pkl"
with open(data_path, 'rb') as f:
    df = pickle.load(f)
print(df.shape)

(1294962, 33)


Outlieri i dalje nisu gotovi, automatska detekcija pomocu IsolationForest ili LocalOutlierFactor mi nije davala dobre rezultate.

Zasad sam samo s desne strane krivulje rucno odbacio vrijednosti vece od q3+1.5iqr. 

Kako upariti 

In [ ]:
df = df[~df['Invoiced price'].isnull()]

# df = df[~df['GM%'].isnull()]
# df = df[df['GM%'] >= -1]  # ???
# df = df[df['GM%'] <= 1]  # ???
#df.dropna(inplace=True)

#0 je po meni legalna vrijednost za invoiced quantity, ali ne i za ordered
df = df[df['Invoiced qty (shipped)'] >= 0]
df = df[df['Invoiced qty (shipped)'] < 1000000]
df = df[df['Ordered qty'] > 0]
df = df[df['Ordered qty'] < 1000000]
df = df[df['Invoiced price'] > 0]
df = df[df['Invoiced price'] < 100000]
df = df[df['Cost of part'] >= 0]
df = df[df['Cost of part'] < 75000]

df = df[~df['# of unique products on a quote'].isnull()]
df = df[df['# of unique products on a quote'] < 374]
df = df[df['# of unique products on a quote'] > 0]

#simple drop
df = df[~df['Product group'].isnull()]
df = df[~df['Manufacturing Location Code'].isnull()]
df = df[~df['Make vs Buy'].isnull()]
df = df[~df['Customer industry'].isnull()]

#ovo nije dobro, maknuo sam ih jer ih je malo. Jel se ovdje moze koristiti SMOTE?
# df = df[df['Product family']!='PC010']
# df = df[df['Product family']!='PC001']
# df = df[df['Product family']!='PC016']

# df[['Material cost of part',
#     'Labor cost of part',
#     'Overhead cost of part'
#     ]] = df[['Material cost of part',
#              'Labor cost of part',
#              'Overhead cost of part'
#              ]].fillna(value=0)
# df['sum'] = df['Material cost of part'] + df['Labor cost of part'] + df['Overhead cost of part']
# df['sum'] = np.where(df['sum']==0, df['Cost of part'], df['sum'])

df['Invoiced qty (shipped)'].astype(int)
df['Ordered qty'].astype(int)
df['# of unique products on a quote'].astype(int)

df['Invoiced qty (shipped)'] = np.log10(df['Invoiced qty (shipped)'] + 1)
df['Ordered qty'] = np.log10(df['Ordered qty'] + 1)
df['Cost of part'] = np.log10(df['Cost of part'] + 1)
df['Invoiced qty (shipped)2'] = np.log10(df['Invoiced qty (shipped)'] + 1)*2
df['Ordered qty2'] = np.log10(df['Ordered qty'] + 1)*2
df['Cost of part2'] = np.log10(df['Cost of part'] + 1)*2
df['Invoiced qty (shipped)3'] = np.log10(df['Invoiced qty (shipped)'] + 1)*3
df['Ordered qty3'] = np.log10(df['Ordered qty'] + 1)*3
df['Cost of part3'] = np.log10(df['Cost of part'] + 1)*3
df['Invoiced price'] = np.log10(df['Invoiced price'] + 1)#????????????????????????????????????????????????
df['# of unique products on a quote'] = np.log10(df['# of unique products on a quote'] + 1)

In [ ]:
to_drop_too_much_missing = ['Price last modified date in the ERP',
                            'Sales Channel - Grouping',
                            'Material cost of part',
                            'Labor cost of part',
                            'Overhead cost of part']

df.drop(to_drop_too_much_missing, inplace=True, axis=1)

to_drop_unimportant = [
    'Sales Channel - Internal',
    'Sales Channel - External']

df.drop(to_drop_unimportant, inplace=True, axis=1)

In [ ]:
to_drop_output = [#'Invoiced price',
                  'GM%']

df.drop(to_drop_output, inplace=True, axis=1)

Na datumima jos uvijek radim, glavna ideja mi je za feature uzeti broj dana izmedju 'Customer First Invoice Date',
                 'Born on date',
                 'Order Date' 
i  Invoice Date-a

In [ ]:
to_drop_highly_correlated = [#'Invoiced price',
                             'Invoiced price (TX)']

df.drop(to_drop_highly_correlated, inplace=True, axis=1)

MS sin cos transformation


In [ ]:
from datetime import datetime
import calendar

def transform_order_date(order_date):
    #order_date = order_date.apply(datetime.fromisoformat)
    day_of_week = []
    day_of_month = []
    day_of_year = []
    for date in order_date:
        dow = date.weekday()
        date_day = date.day-1
        doy = date.timetuple().tm_yday - 1 # starts from 1
        month_length = calendar.monthrange(date.year, date.month)[1]
        year_length = 365 if not calendar.isleap(date.year) else 366
        day_of_week.append(2*np.pi*dow / 7)
        day_of_month.append(2*np.pi*date_day / month_length)
        day_of_year.append(2*np.pi*doy / year_length)
    dow = np.array(day_of_week)
    dom = np.array(day_of_month)
    doy = np.array(day_of_year)
    return np.cos(dow), np.sin(dow), np.cos(dom), np.sin(dom), np.cos(doy), np.sin(doy)

In [ ]:
df['Customer First Invoice Date']=pd.to_datetime(df['Customer First Invoice Date'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
df['Born on date']=pd.to_datetime(df['Born on date'], errors='coerce', format='%Y-%m-%d')
df['Invoice Date']=pd.to_datetime(df['Invoice Date'], errors='coerce', format='%Y-%m-%d')
df['Order Date']=pd.to_datetime(df['Order Date'], errors='coerce', format='%Y-%m-%d')

In [ ]:
df=df[np.isfinite(df['Customer First Invoice Date'])]
df=df[np.isfinite(df['Born on date'])]
df=df[np.isfinite(df['Invoice Date'])]
df=df[np.isfinite(df['Order Date'])]

#print('izvedeni natovi')
df['inv_firinv_diff']= df['Invoice Date'] - df['Customer First Invoice Date']
df['inv_born_diff']= df['Invoice Date'] - df['Born on date']
df['inv_ord_diff']= df['Invoice Date'] - df['Order Date']
# print(df1[~np.isfinite(df1['inv_firinv_diff'])])
# print(df1[~np.isfinite(df1['inv_born_diff'])])
# print(df1[~np.isfinite(df1['inv_ord_diff'])])

#df.drop(to_drop_dates, inplace=True, axis=1)

df['inv_firinv_diff']=df['inv_firinv_diff'].dt.days.astype(int)
df['inv_born_diff']=df['inv_born_diff'].dt.days.astype(int)
df['inv_ord_diff']=df['inv_ord_diff'].dt.days.astype(int)


In [ ]:
date = df['Order Date']
dow_x, dow_y, dom_x, dom_y, doy_x, doy_y = transform_order_date(date)
df['1dow_x'] = dow_x
df['1dow_y'] = dow_y
df['1dom_x'] = dom_x
df['1dom_y'] = dom_y
df['1doy_x'] = doy_x
df['1doy_y'] = doy_y

date = df['Invoice Date']
dow_x, dow_y, dom_x, dom_y, doy_x, doy_y = transform_order_date(date)
df['2dow_x'] = dow_x
df['2dow_y'] = dow_y
df['2dom_x'] = dom_x
df['2dom_y'] = dom_y
df['2doy_x'] = doy_x
df['2doy_y'] = doy_y

date = df['Born on date']
dow_x, dow_y, dom_x, dom_y, doy_x, doy_y = transform_order_date(date)
df['3dow_x'] = dow_x
df['3dow_y'] = dow_y
df['3dom_x'] = dom_x
df['3dom_y'] = dom_y
df['3doy_x'] = doy_x
df['3doy_y'] = doy_y

date = df['Customer First Invoice Date']
dow_x, dow_y, dom_x, dom_y, doy_x, doy_y = transform_order_date(date)
df['4dow_x'] = dow_x
df['4dow_y'] = dow_y
df['4dom_x'] = dom_x
df['4dom_y'] = dom_y
df['4doy_x'] = doy_x
df['4doy_y'] = doy_y

In [ ]:
to_drop_dates = ['Customer First Invoice Date',
                 'Born on date',
                 'Invoice Date',
                 'Order Date']

df.drop(to_drop_dates, inplace=True, axis=1)

Prvih 6 featura u listi su svojevrsni id-evi, koristit ce kod bilo kakvog grupiranja i punjenja praznih vrijednosti(na tom radim).

Zadnja 4 su kategoricki koje jos nisam uklopio. Postoji hashing trick, ali sumnjam obzirom na mali broj klasa da je to najbolje rjesenje.

In [ ]:
to_drop_need_fe = [
                   'Invoice Line #',#ima date u sebi?
                   'Order Line #',
                   'Invoice #',
                   'Order #',
                   '# of unique products on a quote'#droppam svejedno jer je losa korelacija
                   ]

df.drop(to_drop_need_fe, inplace=True, axis=1)

Ideje za neke nove feature

iz worda:

Grupirati po transakcijama

Stvari koje se narucuju u grupama - prihvatljiviji manji GM%

Relationship between order, invoice and deal

Usporedba razlicitih salesamana

Clustering customers

Kako se mjenja cjena itema kroz vrijeme


In [ ]:
#df['itemqty0'] = df.groupby('Order #')['Item Code'].transform('nunique')
df['itemqty_fb'] = df.groupby(['CustomerID', 'Customer industry', 'Manufacturing Region', 'Product group'])[
    'Item Code'].transform('nunique')
df['total_part_cost']=df['Invoiced qty (shipped)']*df['Cost of part']
df['deal_size'] = df.groupby(['CustomerID', 'Customer industry', 'Manufacturing Region', 'Product group'])[
    'total_part_cost'].transform('sum')
df['deal_qty'] = df.groupby(['CustomerID', 'Customer industry', 'Manufacturing Region', 'Product group'])[
    'Invoiced qty (shipped)'].transform('sum')
  
df['meancop'] = df.groupby(['Item Code'])[
    'Cost of part'].transform('mean')

df['cop0'] = df['Cost of part'] == 0
df['meancop'] = np.log10(df['meancop'] + 1)

df['itemqty_fb'] = np.log10(df['itemqty_fb'] + 1)
df['deal_size'] = np.log10(df['deal_size'] + 1)
df['deal_qty'] = np.log10(df['deal_qty'] + 1)
df['itemqty_fb2'] = np.log10(df['itemqty_fb'] + 1)*2
df['deal_size2'] = np.log10(df['deal_size'] + 1)*2
df['deal_qty2'] = np.log10(df['deal_qty'] + 1)*2
df['itemqty_fb3'] = np.log10(df['itemqty_fb'] + 1)*3
df['deal_size3'] = np.log10(df['deal_size'] + 1)*3
df['deal_qty3'] = np.log10(df['deal_qty'] + 1)*3

In [ ]:
to_drop_need_fe2 = ['CustomerID',
                   'Item Code',
                  #  'Product group',
                  #  'Customer industry'
                  #   'Manufacturing Location Code',
                  #  'Make vs Buy',
                   ]

df.drop(to_drop_need_fe2, inplace=True, axis=1)

In [ ]:
# dummies = ['Manufacturing Region',
#            'Intercompany',
#            'Customer Region',
#            'Product family',
#            'Top Customer Group',
            #'Product group',
            #'Manufacturing Location Code'
            # 'Make vs Buy',
            # 'Customer industry'


# df.drop(dummies, inplace=True, axis=1)
df = pd.get_dummies(df, drop_first=True)

Mozda ce trebat dvoklik na mapu Files gore lijevo da se refresha, downloadajte html i otvorite s browserom.

In [ ]:
sv_report = sv.analyze(df, target_feat='Invoiced price')
sv_report.show_html('SV_report_lumen21.html')

                                             |          | [  0%]   00:00 -> (? left)

Report SV_report_lumen21.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


Jel vrijedi tvrdnja:
Sto je dobro za linearnu regresiju dobro je za sve modele?
Tj. moze li nam ovo posluzit za evaluaciju u feature engineeringu prije nego uopce krenemo razmisljati o modelu?

In [ ]:
# to_drop_too_much_missing = ['Cost of part']

# df.drop(to_drop_too_much_missing, inplace=True, axis=1)

In [ ]:
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(df.drop('Invoiced price', axis=1), df['Invoiced price'], test_size=0.25)


# regr2 = DecisionTreeRegressor(max_depth=10)
# regr2.fit(x_train, y_train)
# y_trpred=regr2.predict(x_train)
# y_pred=regr2.predict(x_valid)

regr2 = LinearRegression()
regr2.fit(x_train, y_train)
y_trpred=regr2.predict(x_train)
y_pred=regr2.predict(x_valid)

mse_tr=metrics.mean_squared_error(y_trpred,y_train)
rmse_tr=np.sqrt(mse_tr)
r2_tr=regr2.score(x_train, y_train)
#print(mse_tr)
print(rmse_tr)
print(r2_tr)
#print(r2_tr-rmse_tr)
print()
mse=metrics.mean_squared_error(y_pred,y_valid)
rmse=np.sqrt(mse)
r2=regr2.score(x_valid, y_valid)
#print(mse)
print(rmse)
print(r2)
#print(r2-rmse)

# 0.3726835151106834
# 0.808589594406683

# 0.37093042883107574
# 0.8106338319300423

# 0.33185182763802074
# 0.8399602338321936

# 0.32953367974510733
# 0.8416536212557204

# 0.3229132547126829
# 0.848089292879383

# 0.3227261763839638
# 0.8492521424555781

#all cats no bucket, onehot
# 0.29235276513317354
# 0.8758120089412094

# 0.2925961341055447
# 0.8749992404497785

#binning naivni
# 0.30443668914611455
# 0.8651807379901655

# 0.30593290610581186
# 0.8638108452520514

#cop==0 drop
# 0.1515194040054661
# 0.9661172140479719

# 0.15055867709736637
# 0.9663013643005097

#cop==0 drop, bez cop, sa normalizacijom sume
# 0.16987587941945334
# 0.9572877618352114

# 0.1700816994459431
# 0.9573649230906643

#cop==0 drop, bez cop, bez normalizacijom sume
# 0.17191370487142052
# 0.9563696469654851

# 0.17232038485341084
# 0.9558938325470254

#cop==0 NO drop, bez cop, bez normalizacijom sume
# 0.2805899825432754
# 0.8856227491345126

# 0.2806538862697492
# 0.8849388022639999

#cop==0 NO drop, bez cop, sa normalizacijom sume
# 0.28032561695425884
# 0.8857037981846971

# 0.27988584032207686
# 0.885972878040155



#dcr 10,cop==0 NO drop, bez cop, sa normalizacijom sume
# 0.1562610625023906
# 0.9643865448600812

# 0.1596859653472698
# 0.9631885887880433


#dcr 10,cop==0 drop, sa normalizacijom sume
# 0.12333852220624146
# 0.977479766780645

# 0.1287427479890434
# 0.9755859073315504


#dcr 10,cop==0 drop, bez sume
# 0.12321286552789817
# 0.9775586781791111

# 0.12843762380582355
# 0.9755943379188017

#old
# 0.27216905105109546
# 0.8921496139078924

# 0.2725410030784752
# 0.8922033412285926

#cop==0 feature
# 0.2570651983362088
# 0.903870338844236

# 0.2563229657482765
# 0.9044059053452455


#cop==0 feature and MS order date
# 0.25659169602743553
# 0.9041899684586555

# 0.25744890450639146
# 0.9036679946818357

# svi ms datumi
# 0.25691929763605514
# 0.9054640483813864

# 0.2572879784033347
# 0.905153701512807


# svi datumi
# 0.2565479553943206
# 0.9057201484802678

# 0.2551181190714019
# 0.9067968683965303

0.25671158844893416
0.9056254301938095

0.25423088579749237
0.9073686915743546
